# Technical Factor - Directional Movement (SRDM)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        IF((high + low) <= (m_lag(high, 1) + m_lag(low, 1)), 0, GREATEST(ABS(high - m_lag(high, 1)), ABS(low - m_lag(low, 1)))) AS DMP,
        IF((high + low) >= (m_lag(high, 1) + m_lag(low, 1)), 0, GREATEST(ABS(high - m_lag(high, 1)), ABS(low - m_lag(low, 1)))) AS DMN,
        m_avg(DMP, 10) AS ADMP,
        m_avg(DMN, 10) AS ADMN,
        IF(ADMP > ADMN, (ADMP - ADMN) / ADMP, IF(ADMP = ADMN, 0, (ADMP - ADMN) / ADMN)) AS SRDM,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(SRDM > 0 AND m_lag(SRDM, 1) <= 0, 1, 0) AS TRBY1,
        IF(SRDM < 0 AND m_lag(SRDM, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(SRDM > 0 AND (SRDM - m_lag(SRDM, 1)) > 0, 1, 0) AS MTBY1,
        IF(SRDM < 0 AND (SRDM - m_lag(SRDM, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(SRDM < -0.3 AND (SRDM - m_lag(SRDM, 1)) > 0, 1, 0) AS RVBY1,
        IF(SRDM > 0.3 AND (SRDM - m_lag(SRDM, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(SRDM, 1) <= 0.2 AND SRDM > 0.2, 1, 0) AS BKBY1,
        IF(m_lag(SRDM, 1) >= -0.2 AND SRDM < -0.2, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        DMP,
        DMN,
        ADMP,
        ADMN,
        SRDM,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,DMP,DMN,ADMP,ADMN,SRDM,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-16,600519.SH,0.000000,146.707849,52.956565,107.077994,-0.505439,0,0,0,0,0,0,0,0
1,2025-01-17,600519.SH,0.000000,176.440423,52.956565,100.691601,-0.474072,0,0,0,0,1,0,0,0
2,2025-01-20,600519.SH,234.520765,0.000000,76.408641,72.824441,0.046908,1,0,1,0,0,0,0,0
3,2025-01-21,600519.SH,0.000000,63.456643,76.408641,70.396959,0.078678,0,0,1,0,0,0,0,0
4,2025-01-22,600519.SH,0.000000,219.939585,76.408641,82.338864,-0.072022,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,2025-12-25,600519.SH,110.824535,0.000000,55.313585,24.766890,0.552246,0,0,0,0,0,1,0,0
229,2025-12-26,600519.SH,72.813071,0.000000,51.789360,24.766890,0.521776,0,0,0,0,0,1,0,0
230,2025-12-29,600519.SH,0.000000,76.022928,39.339326,32.369182,0.177180,0,0,0,0,0,0,0,0
231,2025-12-30,600519.SH,0.000000,126.704880,39.339326,35.729138,0.091770,0,0,0,0,0,0,0,0
